In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from tqdm import tqdm
tqdm.pandas()

In [2]:
data_x = np.load('./raw_data/train/y_smp_train.npy')
data_y = np.load('./raw_data/train/pars_smp_train.npy')

In [3]:
data_x.shape, data_y.shape

((1000000, 200, 3), (1000000, 15, 1))

In [4]:
# создаем признаки для обучения 
# Параметр N - количество наборов данных из большого файла 
def create_X(n: int) -> pd.DataFrame:
    global data_x
    # генерируем название столбцов. Это пригодится, если понадобиться делать срезы по столбцам
    cols = list()
    for i in range(1, 201):
        col = f'x{i}'
        cols.append(col)
    # создаем пустой датафрейм
    result = pd.DataFrame(columns=cols)
    for i in tqdm(range(n)):
        # выбираем из большого файла один семпл
        temp_x = data_x[i]
        temp_x = pd.DataFrame(temp_x)
        # транспонируем данные
        temp_x = temp_x.T
        temp_x.columns = cols
        # обединяем результирующий датафрейм с датафреймом одного сэмпла
        result = pd.concat([result, temp_x])
    result = result.reset_index(drop=True)
    return result
    

In [78]:
# создаем таргет для обучения 
# Параметр N - количество наборов данных из большого файла 
# Параметр num - номер целевого параметра от 0 до 14 
def create_y(n: int, num: int) -> pd.DataFrame:
    global data_y
    # создаем пустой датафрейм
    result = pd.DataFrame(columns=['y'])
    for i in tqdm(range(n)):
        temp_y = list()
        temp = data_y[i][num]
        for _ in range(3):
            temp_y.append(temp)
        temp_y = pd.DataFrame(temp_y, columns=['y'])
        # обединяем результирующий датафрейм с датафреймом одного сэмпла
        result = pd.concat([result, temp_y])
    result = result.reset_index(drop=True)
    return result    



In [79]:
X = create_X(10000)
y = create_y(10000, 0)

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\3924833446.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_x])
  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\340263733.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_y])
100%|██████████| 10000/10000 [00:01<00:00, 5592.92it/s]


In [7]:
X.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200
0,1.142330,-0.484040,0.665750,0.583209,2.084426,0.401561,0.168327,-0.454143,0.478470,0.438122,...,0.643380,0.157000,-0.749465,-0.782891,2.025607,-1.170654,-0.042919,-0.756649,1.665564,2.243377
1,0.874308,0.619367,0.672932,0.702434,0.785465,0.790633,0.803624,0.910571,0.822872,0.710248,...,1.003370,1.089460,0.974689,0.813481,1.049336,0.886007,0.699045,0.621691,0.925140,1.166110
2,1.446090,1.179399,1.148514,1.185959,1.530353,1.524608,1.431334,1.466774,1.593097,1.426552,...,1.780345,1.916275,1.755813,1.438399,1.866520,1.576825,1.392718,1.248129,1.539656,2.099138
3,4.859037,-7.290394,3.077796,2.585303,-3.253199,3.155431,-1.119519,5.992974,-3.203835,-4.465383,...,-2.039446,0.620485,-0.962892,-3.054348,-1.450241,2.277263,-5.055258,-2.976247,2.930485,-0.620153
4,3.231265,-5.056212,-2.778169,1.032455,-2.054056,1.466364,0.847127,6.731718,2.465817,-4.336061,...,-0.037256,-0.731378,-2.201360,-4.900962,-5.015859,-0.592880,-3.026917,-4.759956,0.838191,2.486871


In [8]:
y.head()

,y
0,0.753449
1,0.753449
2,0.753449
3,1.401436
4,1.401436


In [110]:
# full_df = X.join(y)

In [111]:
# full_df.head()

In [9]:
X.shape, y.shape

((120000, 200), (120000, 1))

In [10]:
X_train = X[:100000]
X_val = X[100000:]
y_train = y[:100000]
y_val = y[100000:]

In [11]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((100000, 200), (20000, 200), (100000, 1), (20000, 1))

Линейная регрессия

In [112]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [12]:
reg = LinearRegression()
reg.fit(X_train, y_train)
reg_pred = reg.predict(X_val)

In [13]:
print('MAE: ', mean_absolute_error(y_val, reg_pred))
print('MSE:', mean_squared_error(y_val, reg_pred))
print('MAPE: ', mean_absolute_percentage_error(y_val, reg_pred))
print('mean y:', y_val.mean().values[0])
print('mean pred:', reg_pred.mean())

MAE:  0.2924011559278072
MSE: 0.13414832438302862
MAPE:  0.2332479975680495
mean y: 1.5002772472939288
mean pred: 1.503743041251569


CatBoost

In [163]:
cat_reg = CatBoostRegressor(random_seed=42, verbose=200, depth=10, eval_metric='RMSE', n_estimators=1000)

In [164]:
cat_reg.fit(X_train, y_train)


Learning rate set to 0.068085
0:	learn: 0.3725729	total: 395ms	remaining: 6m 34s
200:	learn: 0.2711749	total: 1m 18s	remaining: 5m 11s
400:	learn: 0.1971644	total: 2m 31s	remaining: 3m 46s
600:	learn: 0.1490123	total: 3m 41s	remaining: 2m 26s
800:	learn: 0.1156369	total: 4m 48s	remaining: 1m 11s
999:	learn: 0.0923306	total: 5m 51s	remaining: 0us


In [165]:
cat_preds = cat_reg.predict(X_val)

In [166]:
print('MAE: ', mean_absolute_error(y_val, cat_preds))
print('MSE:', mean_squared_error(y_val, cat_preds))
print('MAPE: ', mean_absolute_percentage_error(y_val, cat_preds))
print('mean y:', y_val.mean().values[0])
print('mean pred:', cat_preds.mean())

MAE:  0.2952581420860547
MSE: 0.13346377738802795
MAPE:  0.21587042275389062
mean y: 1.5200868117372108
mean pred: 1.5023822401410203


Цикл на все 15 параметров с логгированием результатов

In [15]:
for i in range(15):
    y = create_y(40000, i)
    y_train = y[:100000]
    y_val = y[100000:]

    reg = LinearRegression()
    reg.fit(X_train, y_train)
    reg_pred = reg.predict(X_val)
    with open(f'./logs/lr_raw_40k.txt', 'a') as f:
        f.write(f'Target param: {i}\n')
        f.write(f'MAE: {mean_absolute_error(y_val, reg_pred)}\n')
        f.write(f'MSE: {mean_squared_error(y_val, reg_pred)}\n')
        f.write(f'MAPE:  {mean_absolute_percentage_error(y_val, reg_pred)}\n')
        f.write(f'mean y: {y_val.mean().values[0]}\n')
        f.write(f'mean pred: {reg_pred.mean()}\n')
        f.write('\n')

  0%|          | 0/40000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\340263733.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_y])
  0%|          | 0/40000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\340263733.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_y])
  0%|          | 0/40000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\340263733.py:15: FutureWarn

PCA-3

In [31]:
# создаем признаки для обучения 
# Параметр N - количество наборов данных из большого файла 
def create_X_PCA_3(n: int) -> pd.DataFrame:
    global data_x
    # Создаем экзмепляр класса PCA
    pca = PCA(n_components=3)
    # генерируем название столбцов. Это пригодится, если понадобиться делать срезы по столбцам
    cols = list()
    for i in range(1, 201):
        col = f'x{i}'
        cols.append(col)
    # создаем пустой датафрейм
    result = pd.DataFrame(columns=cols)
    for i in tqdm(range(n)):
        # выбираем из большого файла один семпл
        temp_x = data_x[i]
        temp_x = pca.fit_transform(temp_x)
        temp_x = pd.DataFrame(temp_x)
        # PCA-трансформация
        # транспонируем данные
        temp_x = temp_x.T
        temp_x.columns = cols
        # обединяем результирующий датафрейм с датафреймом одного сэмпла
        result = pd.concat([result, temp_x])
    result = result.reset_index(drop=True)
    return result

In [32]:
X = create_X_PCA_3(10000)
y = create_y(10000, 0)


  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\925903268.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_x])
  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\340263733.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_y])
100%|██████████| 10000/10000 [00:01<00:00, 5456.43it/s]


In [47]:
X.shape, y.shape

((20000, 200), (20000, 1))

In [48]:
X_train = X[:17000]
X_val = X[17000:]
y_train = y[:17000]
y_val = y[17000:]

In [49]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [50]:
reg = LinearRegression()
reg.fit(X_train, y_train)
reg_pred = reg.predict(X_val)

In [51]:
print('MAE: ', mean_absolute_error(y_val, reg_pred))
print('MSE:', mean_squared_error(y_val, reg_pred))
print('MAPE: ', mean_absolute_percentage_error(y_val, reg_pred))
print('mean y:', y_val.mean().values[0])
print('mean pred:', reg_pred.mean())

MAE:  0.2975884800269682
MSE: 0.13544867845663236
MAPE:  0.21811343233365627
mean y: 1.5210943708504256
mean pred: 1.503262902256717


In [53]:
for i in range(15):
    y = create_y_PCA_2(10000, i)
    y_train = y[:17000]
    y_val = y[17000:]

    reg = LinearRegression()
    reg.fit(X_train, y_train)
    reg_pred = reg.predict(X_val)
    with open(f'./logs/lr_PCA2_10k.txt', 'a') as f:
        f.write(f'Target param: {i}\n')
        f.write(f'MAE: {mean_absolute_error(y_val, reg_pred)}\n')
        f.write(f'MSE: {mean_squared_error(y_val, reg_pred)}\n')
        f.write(f'MAPE:  {mean_absolute_percentage_error(y_val, reg_pred)}\n')
        f.write(f'mean y: {y_val.mean().values[0]}\n')
        f.write(f'mean pred: {reg_pred.mean()}\n')
        f.write('\n')

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\1979373030.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_y])
  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\1979373030.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_y])
  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\1979373030.py:15: FutureW

PCA-2

In [40]:
# создаем признаки для обучения 
# Параметр N - количество наборов данных из большого файла 
def create_X_PCA_2(n: int) -> pd.DataFrame:
    global data_x
    # Создаем экзмепляр класса PCA
    pca = PCA(n_components=2)
    # генерируем название столбцов. Это пригодится, если понадобиться делать срезы по столбцам
    cols = list()
    for i in range(1, 201):
        col = f'x{i}'
        cols.append(col)
    # создаем пустой датафрейм
    result = pd.DataFrame(columns=cols)
    for i in tqdm(range(n)):
        # выбираем из большого файла один семпл
        temp_x = data_x[i]
        temp_x = pca.fit_transform(temp_x)
        temp_x = pd.DataFrame(temp_x)
        # PCA-трансформация
        # транспонируем данные
        temp_x = temp_x.T
        temp_x.columns = cols
        # обединяем результирующий датафрейм с датафреймом одного сэмпла
        result = pd.concat([result, temp_x])
    result = result.reset_index(drop=True)
    return result

In [ ]:
# создаем таргет для обучения 
# Параметр N - количество наборов данных из большого файла 
# Параметр num - номер целевого параметра от 0 до 14 
def create_y_PCA_2(n: int, num: int) -> pd.DataFrame:
    global data_y
    # создаем пустой датафрейм
    result = pd.DataFrame(columns=['y'])
    for i in tqdm(range(n)):
        temp_y = list()
        temp = data_y[i][num]
        for _ in range(2):
            temp_y.append(temp)
        temp_y = pd.DataFrame(temp_y, columns=['y'])
        # обединяем результирующий датафрейм с датафреймом одного сэмпла
        result = pd.concat([result, temp_y])
    result = result.reset_index(drop=True)
    return result 

In [41]:
X = create_X_PCA_2(10000)
y = create_y_PCA_2(10000, 0)

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\2815727293.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_x])
  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\340263733.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_y])
100%|██████████| 10000/10000 [00:01<00:00, 5078.85it/s]


In [42]:
X.shape, y.shape

((20000, 200), (30000, 1))

In [ ]:
X_train = X[:25000]
X_val = X[25000:]
y_train = y[:25000]
y_val = y[25000:]

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)
reg_pred = reg.predict(X_val)

In [ ]:
print('MAE: ', mean_absolute_error(y_val, reg_pred))
print('MSE:', mean_squared_error(y_val, reg_pred))
print('MAPE: ', mean_absolute_percentage_error(y_val, reg_pred))
print('mean y:', y_val.mean().values[0])
print('mean pred:', reg_pred.mean())

3x200

In [54]:
# создаем признаки для обучения 
# Параметр N - количество наборов данных из большого файла 
def create_X_3x200(n: int) -> pd.DataFrame:
    global data_x
    # генерируем название столбцов. Это пригодится, если понадобиться делать срезы по столбцам
    cols = list()
    for i in range(1, 4):
        col = f'x{i}'
        cols.append(col)
    # создаем пустой датафрейм
    result = pd.DataFrame(columns=cols)
    for i in tqdm(range(n)):
        # выбираем из большого файла один семпл
        temp_x = data_x[i]
        temp_x = pd.DataFrame(temp_x)
        temp_x.columns = cols
        # обединяем результирующий датафрейм с датафреймом одного сэмпла
        result = pd.concat([result, temp_x])
    result = result.reset_index(drop=True)
    return result

In [58]:
# создаем таргет для обучения 
# Параметр N - количество наборов данных из большого файла 
# Параметр num - номер целевого параметра от 0 до 14 
def create_y_3x200(n: int, num: int) -> pd.DataFrame:
    global data_y
    # создаем пустой датафрейм
    result = pd.DataFrame(columns=['y'])
    for i in tqdm(range(n)):
        temp_y = list()
        temp = data_y[i][num]
        for _ in range(200):
            temp_y.append(temp)
        temp_y = pd.DataFrame(temp_y, columns=['y'])
        # обединяем результирующий датафрейм с датафреймом одного сэмпла
        result = pd.concat([result, temp_y])
    result = result.reset_index(drop=True)
    return result  

In [57]:
X = create_X_3x200(10000)


  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\2748121111.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_x])
100%|██████████| 10000/10000 [01:11<00:00, 139.90it/s]


NameError: name 'create_y_3x200' is not defined

In [59]:
y = create_y_3x200(10000, 0)

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\wilte\AppData\Local\Temp\ipykernel_19784\1399651710.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, temp_y])
100%|██████████| 10000/10000 [00:38<00:00, 259.30it/s]


In [60]:
X.shape, y.shape

((2000000, 3), (2000000, 1))

In [61]:
X_train = X[:1750000]
X_val = X[1750000:]
y_train = y[:1750000]
y_val = y[1750000:]

In [62]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1750000, 3), (250000, 3), (1750000, 1), (250000, 1))

In [63]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [64]:
reg = LinearRegression()
reg.fit(X_train, y_train)
reg_pred = reg.predict(X_val)

In [65]:
print('MAE: ', mean_absolute_error(y_val, reg_pred))
print('MSE:', mean_squared_error(y_val, reg_pred))
print('MAPE: ', mean_absolute_percentage_error(y_val, reg_pred))
print('mean y:', y_val.mean().values[0])
print('mean pred:', reg_pred.mean())

MAE:  0.28933365993470467
MSE: 0.12914183431904555
MAPE:  0.20936789935264474
mean y: 1.527868761112596
mean pred: 1.5033292582346447


In [74]:
# создаем признаки для обучения 
# Параметр N - количество наборов данных из большого файла 
def create_X_np(n: int) -> pd.DataFrame:
    global data_x
    # генерируем название столбцов. Это пригодится, если понадобиться делать срезы по столбцам
    cols = list()
    for i in range(1, 201):
        col = f'x{i}'
        cols.append(col)
    # создаем пустой датафрейм
    #result = pd.DataFrame(columns=cols)
    result = data_x[0]
    result = result.T
    for i in tqdm(range(1, n)):
        # выбираем из большого файла один семпл
        temp_x = data_x[i].T    
        result = np.vstack((result, temp_x))
    result = pd.DataFrame(result, columns=cols)
    result = result.reset_index(drop=True)
    return result

In [77]:
a = create_X_np(10000)

100%|██████████| 9999/9999 [01:20<00:00, 123.66it/s] 


In [81]:
a.shape

(30000, 200)